In [1]:
%load_ext dotenv
%dotenv

In [2]:
from VLA_LLM import prompts
from VLA_LLM.agents import ChatConversationalVLAAgent
from VLA_LLM.api import get_community_info
from VLA_LLM.community_info import community_dict_to_prompt
from VLA_LLM.tools import AppointmentAvailabilityTool
from VLA_LLM.tools import AppointmentSchedulerTool
from VLA_LLM.tools import AppointmentSchedulerAndAvailabilityTool
from VLA_LLM.tools import CurrentTimeTool

In [3]:
# most deterministic results
temperature = 0

In [4]:
# hardcode these for now
client_id = 7597721
group_id = 40
api_key = '43939788d50b47c1a34c32ae56b02a0e'

In [5]:
tools = [CurrentTimeTool(), AppointmentSchedulerAndAvailabilityTool(client_id=client_id, group_id=group_id, api_key=api_key)]
# tools = [CurrentTimeTool(), AppointmentSchedulerTool(), AppointmentAvailabilityTool()]

In [6]:
community_info = get_community_info(90)

In [7]:
community_info_prompt = community_dict_to_prompt(community_info)

In [8]:
print(community_info_prompt)

Building name: Downtown Commons
Location: 123 West Street, New York, NY 10011
Parking options:
  1. Parking type: Parking Available
     Parking fee: $100.0
     Parking fee term: Monthly
     Parking notes: noties parking available
Here is information about whether pets are allowed and how much it costs:
  Policy: No Pets Allowed
  Extra deposit for dogs: 50.0
  Extra deposit for cats: 20.0
  Is deposit refundable?: Yes
  Extra fee needed: 20.0
  Extra rent needed: 100.0
  Maximum number allowed: 2
  Maximum weight allowed: 20
  Are there breed restrictions?: Yes
  Notes about restricted breeds: Pit, Bulls, Rottweilers, Presa, Canario, German, Shepherds, Chowchows, Dobermans, Malamutes, St., Bernards, Great, Danes, Akitas
  Extra notes about pets: Fees depend on the number of pets that you have.
Special offers: There's $100 off the first month!
The following is information on utility companies, whether the utility is paid by the tenant or community, and extra fees:
  Electric: Company

In [9]:
prompt_template = (
    f"{prompts.prompt_two_tool_explicit.format(community_info=community_info_prompt)}\n\n"
    "Here is the prospect message:\n\n{prospect_message}"
)

In [10]:
agent = ChatConversationalVLAAgent(temperature, tools)

In [11]:
def run_agent(message):
    return agent.agent_chain.run(input=prompt_template.format(prospect_message=message))

### Test Conversation 1

Basic questions about the community.

In [12]:
run_agent("what is the name of the building?")



> Entering new AgentExecutor chain...
{
    "action": "Final Answer",
    "action_input": "The name of the building is Downtown Commons."
}

> Finished chain.


'The name of the building is Downtown Commons.'

In [ ]:
run_agent("Are there extra costs for having a dog?")

In [ ]:
run_agent("does the building have a gym?")

In [ ]:
run_agent("do you have any videos i can see?")

In [ ]:
run_agent("does the building have a lounge?")

In [ ]:
run_agent("does the building have a swimming pool?")

In [ ]:
run_agent("What is the parking situation?")

In [ ]:
run_agent("are there any associated leasing costs?")

In [ ]:
run_agent("where is the building located?")

In [ ]:
run_agent("i have a dog. can i bring him?")

In [ ]:
run_agent("What is the parking situation? can i bring my dog as well?")

In [ ]:
run_agent("Are there any specials running right now?")

In [ ]:
run_agent("how much are utilities?")

In [ ]:
run_agent("Do you offer section 8?")

In [ ]:
run_agent("how do i apply?")

In [ ]:
run_agent("do you have any photos of the units?")

### Test Conversation 2

Asking about an available appointment date.

In [ ]:
run_agent("Can I schedule a tour for the 15th?")

In [13]:
run_agent("Can I schedule a tour for 8/15?")



> Entering new AgentExecutor chain...
{
    "action": "get_current_time",
    "action_input": ""
}
Observation: Monday 2023-08-14 13:12:26
Thought:{
    "action": "appointment_scheduler_availability",
    "action_input": "2023-08-15"
}
Observation: The next available appointment times on that date are 2023-08-15T08:00:00, 2023-08-15T08:15:00, 2023-08-15T08:30:00.
Thought:{
    "action": "Final Answer",
    "action_input": "Sure! We have availability for a tour on August 15th. The next available appointment times are 8:00 AM, 8:15 AM, and 8:30 AM. Which time works best for you?"
}

> Finished chain.


'Sure! We have availability for a tour on August 15th. The next available appointment times are 8:00 AM, 8:15 AM, and 8:30 AM. Which time works best for you?'

In [14]:
run_agent("Can I book one for 8am?")



> Entering new AgentExecutor chain...
{
    "action": "get_current_time",
    "action_input": ""
}
Observation: Monday 2023-08-14 13:12:42
Thought:{
    "action": "appointment_scheduler_availability",
    "action_input": "2023-08-15 08:00:00"
}
Observation: Scheduled successfully for 08:00:00 on 2023-08-15!
Thought:{
    "action": "Final Answer",
    "action_input": "Sure! I have scheduled a tour for you on August 15th at 8 AM. We look forward to showing you around Downtown Commons!"
}

> Finished chain.


'Sure! I have scheduled a tour for you on August 15th at 8 AM. We look forward to showing you around Downtown Commons!'

### Test Conversation 3

Asking about an unavailable appointment date.

In [ ]:
run_agent("You do have any available appointments on 7/29")

### Test Conversation 4

Asking about an available appointment time.

In [ ]:
run_agent("Can I book a tour for 7/29 at 1pm?")

### Test Conversation 5

Asking about an available appointment time in relative terms.

In [ ]:
run_agent("Can I book a tour for this Thursday at 1pm?")

### Test Conversation 6

Asking about an unavailable appointment time.

In [ ]:
run_agent("Can I book a tour for 7/30 at 3?")

### Test Conversation 7

Asking about an appointment and an inquiry.

In [ ]:
run_agent("Can I book a tour for 7/29 at 1pm? i have a car - what is the parking situation like?")